In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='white', palette='bright')
plt.rcParams["figure.figsize"] = [7,5]
import pandas_datareader as pdr
from scipy.optimize import minimize
import yfinance as yf

In [49]:
# define market parameters

tickers = ['SPY','TLT','DJP','VTV','MTUM']
start = '2015-09-30'
end = '2022-09-30'

In [50]:
# # create dataframe of prices (uses Yahoo Finance adjusted closing prices)
# This code cell is outdated
# data_raw = pdr.data.DataReader(tickers,'yahoo',start,end)
# data_raw = data_raw.loc[:,'Adj Close']

In [51]:
data_raw = pd.DataFrame()
for ticker in tickers:
    data = yf.download(ticker, start=start, end=end)['Adj Close']
    data_raw[ticker] = data

# data_raw = yf.download(tickers, start=start, end=end)['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [52]:
# create dataframe of returns

returns_raw = data_raw.pct_change(periods=1).dropna()
returns_m = returns_raw.copy().resample('BM').apply(lambda x: (1+x).prod() - 1)


In [53]:
# create function to get risk contributions


def get_risk_contribution(weights, cov):

    portfolio_variance = np.dot(np.dot(cov,weights), weights.T)
    risk_contribution = (np.dot(cov,weights)*weights.T) / portfolio_variance

    return risk_contribution

In [54]:
# create functions to solve for risk parity weights



def target_risk_contributions(target_risk, cov):
'''
finds the min variance portfolio with target risk
'''
    n = cov.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0), ) * n
    normalize_weights = {'type':'eq',
                        'fun':lambda weights: np.sum(weights) - 1}


    def msd_risk(weights, target_risk, cov):

        w0 = get_risk_contribution(weights, cov)
        return ((w0 - target_risk)**2).sum()


    weights = minimize(msd_risk, init_guess,
                       args=(target_risk,cov), method='SLSQP',
                       options = {'disp': False},
                       constraints=(normalize_weights,),
                       bounds=bounds)
    return weights.x


def equal_risk_contributions(cov):

    n=cov.shape[0]
    return target_risk_contributions(target_risk=np.repeat(1/n,n),cov=cov)

IndentationError: expected an indented block (1958702576.py, line 6)

In [9]:
# create equal weight portfolio of factor ETFs

factor_equities = np.sum(np.repeat(1/2,2) * returns_raw.loc[:,'VTV':],axis=1) #daily
factor_equities_m = np.sum(np.repeat(1/2,2) * returns_m.loc[:,'VTV':],axis=1) #monthly

In [10]:
# sub dataframes for analysis


# dataframes containing asset ETF returns daily (raw) and monthly (m)
returns_raw_assets = returns_raw.loc[:,:'DJP']
returns_m_assets = returns_m.loc[:,:'DJP']


# dataframes containing factor ETF returns daily (raw) and monthly (m)
returns_raw_factors = pd.DataFrame({'Factor Equities':factor_equities, 'TLT':returns_raw['TLT'], 'DJP':returns_raw['DJP']})
returns_m_factors = pd.DataFrame({'Factor Equities':factor_equities_m, 'TLT':returns_m['TLT'], 'DJP':returns_m['DJP']})

In [11]:
# calculate weights for risk parity portfolios (asset ETFs)
# uses trailing 252 days for vol calculations at each month end


rp_assets_weights = np.zeros((len(returns_raw),3))
risk_assets_contributions = np.zeros((len(returns_raw),3))

for i in range(len(returns_raw)):
    if returns_raw.index[i] in (returns_m.index):
        rp_assets_weights[i,:] = equal_risk_contributions(returns_raw_assets[i:252+i].cov())
        risk_assets_contributions[i,:] = get_risk_contribution(equal_risk_contributions(returns_raw_assets[i:252+i].cov()),returns_raw_assets[i:252+i].cov())

In [12]:
# calculate weights for risk parity portfolios (factor ETFs)
# uses trailing 252 days for vol calculations at each month end



rp_factor_weights = np.zeros((len(returns_raw),3))
risk_factor_contributions = np.zeros((len(returns_raw),3))

for i in range(len(returns_raw)):
    if returns_raw.index[i] in (returns_m.index):
        rp_factor_weights[i,:] = equal_risk_contributions(returns_raw_factors[i:252+i].cov())
        risk_factor_contributions[i,:] = get_risk_contribution(equal_risk_contributions(returns_raw_factors[i:252+i].cov()),returns_raw_factors[i:252+i].cov())

In [13]:
# remove times when weights are 0

rp_assets_weights = rp_assets_weights[~np.all(rp_assets_weights == 0, axis=1)]
risk_assets_contributions = risk_assets_contributions[~np.all(risk_assets_contributions == 0, axis=1)]

rp_factor_weights = rp_factor_weights[~np.all(rp_factor_weights == 0, axis=1)]
risk_factor_contributions = risk_factor_contributions[~np.all(risk_factor_contributions == 0, axis=1)]

In [37]:
rp_assets_weights[:-9].shape,returns_m_assets[12:].shape

((72, 3), (72, 3))

In [39]:
# calculate unlevered portfolio returns


rp_assets_port = np.sum(rp_assets_weights[:-9] * returns_m_assets[12:],axis=1) #risk parity (assets) portfolio
eq_assets_port = np.sum(np.repeat(1/3,3) * returns_m_assets[12:],axis=1) #equal weight (assets) portfolio
rp_factor_port = np.sum(rp_factor_weights[:-9] * returns_m_factors[12:],axis=1) #risk parity (factors) portfolio

In [40]:
# dataframe containing unlevered portfolio returns


compare_portfolios = \
pd.DataFrame({'Simple Risk Parity':rp_assets_port,'Factor Risk Parity':rp_factor_port,'Equal Weights':eq_assets_port})

In [41]:
# calculate leverage based on trailing 12 month vol


simple_leverage = (compare_portfolios.iloc[:,-1].rolling(12).var() / compare_portfolios.iloc[:,0].rolling(12).var() - 1).dropna()
factor_leverage = (compare_portfolios.iloc[:,-1].rolling(12).var() / compare_portfolios.iloc[:,1].rolling(12).var() - 1).dropna()
leverages = pd.DataFrame({'Simple Risk Parity':simple_leverage,'Factor Risk Parity':factor_leverage})

In [42]:
# calculate levered returns


simple_rp_levered = compare_portfolios.iloc[11:,0] * np.sqrt(1+simple_leverage)
factor_rp_levered = compare_portfolios.iloc[11:,1] * np.sqrt(1+factor_leverage)

In [43]:
# dataframe containing levered portfolio returns



compare_levered_portfolios = \
pd.DataFrame({'Simple Risk Parity (levered)': simple_rp_levered,'Factor Risk Parity (levered)': factor_rp_levered, \
              'Equal Weights':eq_assets_port}).dropna()

In [44]:
# create wealth indices for performance stats



portfolio_indices = compare_levered_portfolios.copy()
portfolio_indices.iloc[0,:] = np.repeat(0,3)
portfolio_indices = 100*(1+portfolio_indices).cumprod()

In [45]:
# calculate high-water mark


hwm = portfolio_indices.cummax()

In [46]:
# calculate drawdowns


drawdowns = portfolio_indices/hwm - 1
max_drawdown = drawdowns.min()
max_drawdown

Simple Risk Parity (levered)   -0.171916
Factor Risk Parity (levered)   -0.176153
Equal Weights                  -0.162105
dtype: float64

In [47]:
# calculate tracking error

excess_returns = compare_levered_portfolios.apply(lambda x: x - x["Equal Weights"],axis=1).iloc[1:,:-1]
tracking_error = excess_returns.std() * np.sqrt(12)
tracking_error

Simple Risk Parity (levered)    0.028112
Factor Risk Parity (levered)    0.031107
dtype: float64